##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TensorFlow Lite Model Maker 进行目标检测

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a>
</td>
</table>

在此 CoLab 笔记本中，您将学习如何使用 [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) 库来训练能够在移动设备上检测图像中的沙拉的自定义目标检测模型。

Model Maker 库使用*迁移学习*来简化使用自定义数据集训练 TensorFlow Lite 模型的过程。使用您自己的自定义数据集重新训练 TensorFlow Lite 模型可以减少所需的训练数据量，并将缩短训练时间。

您将使用公开可用的 *Salad* 数据集，该数据集创建自 [Open Images Dataset V4](https://storage.googleapis.com/openimages/web/index.html)。

该数据集中的每个图像都包含标记为以下其中一类的对象：

- 烘焙食品
- 奶酪
- 沙拉
- 海鲜
- 番茄

该数据集包含指定每个对象所在位置的边界框以及对象的标签。

以下是数据集中的示例图像：

<br>

<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" class="">


## <a>先决条件</a>


### 安装所需的软件包

首先安装所需的包，包括来自 [GitHub 仓库](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)的 Model Maker 软件包和将用于评估的 pycocotools 库。

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

导入所需的软件包。

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### 准备数据集

在这里，您将使用与 AutoML [快速入门](https://cloud.google.com/vision/automl/object-detection/docs/edge-quickstart#preparing_a_dataset)相同的数据集。

*Salads* 数据集可从以下地址获得：`gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv`.

其中包含 175 个用于训练的图像，25 个用于验证的图像，以及 25 个用于测试的图像。数据集有五个类：`Salad`、`Seafood`、`Tomato`、`Baked goods`、`Cheese`。

<br>

数据集以 CSV 格式提供：

```
TRAINING,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Salad,0.0,0.0954,,,0.977,0.957,,
VALIDATION,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Seafood,0.0154,0.1538,,,1.0,0.802,,
TEST,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Tomato,0.0,0.655,,,0.231,0.839,,
```

- 每一行对应于一个定位在较大图像中的对象，每个对象被专门指定为测试、训练或验证数据。在本笔记本的后面阶段，您将了解关于这么做的意义的更多信息。
- 这里包含的三行表示**同一图像中的三个不同对象**，可从以下地址获得：`gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg`。
- 每一行都有不同的标签：`Salad`、`Seafood`、`Tomato` 等。
- 使用左上角和右下角顶点为每个图像指定边界框。

以下是这三行的可视化效果：

<br>

<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" class="">

如果您想了解有关如何准备您自己的 CSV 文件以及创建有效数据集的最低要求的更多信息，请参阅[准备您的训练数据](https://cloud.google.com/vision/automl/object-detection/docs/prepare)指南了解更多详细信息。

如果您是 Google Cloud 的新用户，您可能想知道 `gs://` 网址是什么意思。它们是存储在 [Google Cloud Storage](https://cloud.google.com/storage) (GCS) 上的文件的网址。如果您在 GCS 上公开您的文件或[验证您的客户端](https://cloud.google.com/storage/docs/authentication#libauth)，Model Maker 可以像读取本地文件一样读取这些文件。

然而，您不需要将图片保存在 Google Cloud 上就可以使用 Model Maker。您可以在 CSV 文件中使用本地路径，Model Maker 将正常工作。

## 快速入门

训练目标检测模型有六个步骤：

**第 1 步：选择目标检测模型架构。**

本教程使用 EfficientDet-Lite0 模型。EfficientDet-Lite[0-4] 是一系列移动/物联网友好的目标检测模型，派生自 [EfficientDet](https://arxiv.org/abs/1911.09070) 架构。

以下是每种 EfficientDet-Lite 模型之间的性能对比。

模型架构 | 大小 (MB)* | 延迟 (ms)** | 平均精度***
--- | --- | --- | ---
EfficientDet-Lite0 | 4.4 | 37 | 25.69%
EfficientDet-Lite1 | 5.8 | 49 | 30.55%
EfficientDet-Lite2 | 7.2 | 69 | 33.97%
EfficientDet-Lite3 | 11.4 | 116 | 37.70%
EfficientDet-Lite4 | 19.9 | 260 | 41.96%

<i> * 整数量化模型的大小。<br> ** 延迟在使用 4 个 CPU 线程的 Pixel 4 上测得。<br> *** 平均精度是 COCO 2017 验证数据集上的 mAP（平均精度均值）。</i>


In [ ]:
spec = model_spec.get('efficientdet_lite0')

**第 2 步：加载数据集。**

Model Maker 将接收 CSV 格式的输入数据。使用 `object_detector.DataLoader.from_csv` 方法加载数据集，并将其分割为训练、验证和测试图像。

- 训练图像：这些图像用于训练目标检测模型识别沙拉成分。
- 验证图像：这些是模型在训练过程中没有见过的图像。您将使用它们来决定何时应停止训练，以避免[过拟合](https://en.wikipedia.org/wiki/Overfitting)。
- 测试图像：这些图像用于评估最终模型的性能。

您可以直接从 Google Cloud Storage 加载 CSV 文件，但不需要在 Google Cloud 上保留您的图像来使用 Model Maker。您可以在计算机上指定一个本地 CSV 文件，Model Maker 即可正常工作。

In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

**第 3 步：用训练数据训练 TensorFlow 模型。**

- EfficientDet-Lite0 模型默认使用 `epochs = 50`，这意味着它将对训练数据集进行 50 次遍历。您可以在训练期间查看验证准确率，并提前停止，以避免过拟合。
- 在此处设置 `batch_size = 8`，这样您将看到，遍历训练数据集中的 175 个图像需要 21 个步骤。
- 设置 `train_whole_model=True` 可以对整个模型进行微调，而不仅仅是训练头层来提高准确率。代价是训练模型可能需要更长的时间。

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

**第 4 步：在测试数据上评估模型。**

在使用训练数据集中的图像训练目标检测模型之后，使用测试数据集中的剩余 25 个图像来评估该模型在它以前从未见过的新数据上的性能。

由于默认批次大小为 64，因此只需一个步骤即可遍历测试数据集中的 25 个图像。

评估指标与 [COCO](https://cocodataset.org/#detection-eval) 相同。

In [ ]:
model.evaluate(test_data)

**第 5 步：导出为 TensorFlow Lite 模型**

通过指定要将量化模型导出到的文件夹，将训练的目标检测模型导出为 TensorFlow Lite 格式。默认的训练后量化技术是全整数量化。

In [ ]:
model.export(export_dir='.')

**第 6 步：评估 TensorFlow Lite 模型。**

在导出为 TFLite 时，有几个因素可能会影响模型准确率：

- [量化](https://www.tensorflow.org/lite/performance/model_optimization)有助于将模型大小缩小为原来的四分一直，但代价是准确率会略微下降。
- 原始 TensorFlow 模型使用每个类的[非极大值抑制 (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) 进行后处理，而 TFLite 模型使用全局 NMS，速度快得多，但准确率较低。Keras 最多输出 100 个检测，而 Tflite 最多输出 25 个检测。

因此，您必须评估导出的 TFLite 模型，并将其准确率与原始 TensorFlow 模型进行比较。

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

您可以使用 Colab 的左侧边栏下载 TensorFlow Lite 模型文件。右键点击 `model.tflite` 文件，然后选择 `Download` 将其下载到本地计算机。

可以使用 [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) 的 [ObjectDetector API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) 将此模型集成到 Android 或 iOS 应用中。

有关如何在工作应用中使用模型的更多详细信息，请参阅 [TFLite 目标检测示例应用](https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/android/lib_task_api/src/main/java/org/tensorflow/lite/examples/detection/tflite/TFLiteObjectDetectionAPIModel.java#L91)。

*注：Android Studio Model Binding 目前还不支持目标检测，请使用 TensorFlow Lite Task Library。*

## （可选）在您的图像上测试 TFLite 模型

您可以使用互联网上的图像测试训练后的 TFLite 模型。

- 将下面的 `INPUT_IMAGE_URL` 替换为所需的输入图像。
- 调整 `DETECTION_THRESHOLD` 以更改模型的灵敏度。较低的阈值意味着模型将拾取更多对象，但也会有更多的错误检测。与此同时，更高的阈值意味着该模型将只拾取它确信检测到的对象。

尽管目前需要一些样板代码才能在 Python 中运行模型，但将模型集成到移动应用中只需要几行代码。

In [ ]:
#@title Load the trained TFLite model and define some visualization functions

import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
#@title Run object detection and show the detection results

INPUT_IMAGE_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg" #@param {type:"string"}
DETECTION_THRESHOLD = 0.3 #@param {type:"number"}

TEMP_FILE = '/tmp/image.png'

!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
im = Image.open(TEMP_FILE)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(TEMP_FILE, 'PNG')

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    TEMP_FILE,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)

## （可选）针对 Edge TPU 编译

现在您已经有了量化的 EfficientDet Lite 模型，可以编译并部署到 [Coral EdgeTPU](https://coral.ai/)。

**第 1 步：安装 EdgeTPU 编译器**

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

**第 2 步：选择 Edge TPU 数量，然后编译**

EdgeTPU 有 8MB 的 SRAM 用于缓存模型参数（[更多信息](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching)）。这意味着对于大于 8MB 的模型，为了传递模型参数，推断时间将增加。避免这种情况的一种方式是[模型流水线](https://coral.ai/docs/edgetpu/pipeline/) - 将模型拆分成可以使用专用 EdgeTPU 的段。这可以显著改善延迟。

下表可用作要使用的 Edge TPU 数量的参考 - 由于中间张量无法放入片上内存，较大的模型将无法使用单个 TPU 编译。

模型架构 | 最低 TPU 数 | 建议 TPU 数
--- | --- | ---
EfficientDet-Lite0 | 1 | 1
EfficientDet-Lite1 | 1 | 1
EfficientDet-Lite2 | 1 | 2
EfficientDet-Lite3 | 2 | 2
EfficientDet-Lite4 | 2 | 3

In [ ]:
NUMBER_OF_TPUS =  1#@param {type:"number"}

!edgetpu_compiler model.tflite --num_segments=$NUMBER_OF_TPUS

**第 3 步：下载并运行模型**

经过编译后，现在可以在 EdgeTPU 上运行模型以进行目标检测。首先，使用 Colab 的左侧边栏下载编译后的 TensorFlow Lite 模型文件。右键点击 `model_edgetpu.tflite` 文件，然后选择 `Download` 将其下载到本地计算机。

现在，您可以用您喜欢的方式运行模型。检测示例包括：

- [pycoral 检测](https://github.com/google-coral/pycoral/blob/master/examples/detect_image.py)
- [基础 TFLite 检测](https://github.com/google-coral/tflite/tree/master/python/examples/detection)
- [示例视频检测](https://github.com/google-coral/examples-camera)
- [libcoral C++ API](https://github.com/google-coral/libcoral)


## 高级用法

本部分介绍高级用法主题，如调整模型和训练超参数。

### 加载数据集

#### 加载您自己的数据

您可以上传您自己的数据集以完成本教程。请使用 Colab 的左侧边栏上传您的数据集。

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_object_detection.png" width="1000" hspace="0" alt="上传文件">

如果您不想将数据集上传到云端，也可以按照[指南](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)在本地运行库。

#### 使用不同的数据格式加载数据

Model Maker 库还支持 `object_detector.DataLoader.from_pascal_voc` 方法来加载 [PASCAL VOC](https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5#:~:text=Pascal%20VOC%20is%20an%20XML,for%20training%2C%20testing%20and%20validation) 格式的数据。[makesense.ai](https://www.makesense.ai/) 和 [LabelImg](https://github.com/tzutalin/labelImg) 工具可以注解图像并将注解保存为 PASCAL VOC 数据格式的 XML 文件：

```python
object_detector.DataLoader.from_pascal_voc(image_dir, annotations_dir, label_map={1: "person", 2: "notperson"})
```


### 自定义 EfficientDet 模型超参数

可以调整的模型和训练流水线参数包括：

- `model_dir`：模型检查点文件的保存位置。如果未设置，将使用临时目录。
- `steps_per_execution`：每个训练执行的步骤数。
- `moving_average_decay`浮点。用于维护训练参数的移动平均值的衰减。
- `var_freeze_expr`：映射待冻结变量的前缀名称的正则表达式，表示在训练期间保持不变。更具体地说，在代码库中使用 `re.match(var_freeze_expr, variable_name)` 来映射要冻结的变量。
- `tflite_max_detections`：整数，默认为 25。TFLite 模型中的最大输出检测数。
- `strategy`：指定使用哪种分布策略的字符串。可接受的值为 'tpu'、'gpus'、None。'tpu' 是指使用 TPUStrategy。'gpus' 是指为多 GPU 使用 MirroredStrategy。如果为 None，则使用 OneDeviceStrategy 的 TF 默认值。
- `tpu`：用于训练的 Cloud TPU。这应该是创建 Cloud TPU 时使用的名称，或者是 grpc://ip.address.of.tpu:8470 网址。
- `use_xla`：即使策略不是 TPU，也使用 XLA。如果策略是 TPU，则始终使用 XLA，并且此标志无效。
- `profile`：启用配置文件模式。
- `debug`：启用调试模式。

其他可以调整的参数如 [hparams_config.py](https://github.com/google/automl/blob/df451765d467c5ed78bbdfd632810bc1014b123e/efficientdet/hparams_config.py#L170) 中所示。

例如，您可以设置 `var_freeze_expr='efficientnet'`，这将冻结名称前缀为 `efficientnet` 的变量（默认为 `'(efficientnet|fpn_cells|resample_p6)'`）。这允许模型冻结不可训练的变量，并在训练过程中保持它们的值不变。

```python
spec = model_spec.get('efficientdet_lite0')
spec.config.var_freeze_expr = 'efficientnet'
```

### 更改模型架构

您可以通过更改 `model_spec` 来更改模型架构。例如，将 `model_spec` 更改为 EfficientDet-Lite4 模型。

```python
spec = model_spec.get('efficientdet_lite4')
```

### 调整训练超参数

`create` 函数是 Model Maker 库用于创建模型的驱动函数。`model_spec` 参数定义模型规范。目前支持 `object_detector.EfficientDetSpec` 类。`create` 函数包括以下步骤：

1. 根据 `model_spec` 创建用于目标检测的模型。
2. 训练模型。默认周期和默认批次大小由 `model_spec` 对象中的 `epochs` 和 `batch_size` 变量设置。您还可以调整训练超参数，如影响模型准确率的 `epochs` 和 `batch_size`。例如，

- `epochs`：整数，默认为 50。更多周期可以获得更好的准确率，但可以会导致过拟合。
- `batch_size`：整数，默认为 64。一个训练步骤中要使用的样本数。
- `train_whole_model`：布尔值，默认为 False。如果为 true，则训练整个模型。否则，只训练不匹配 `var_freeze_expr` 的层。

例如，您可以使用较少的周期进行训练，并且只使用头层。您可以增加周期数以获得更好的效果。

```python
model = object_detector.create(train_data, model_spec=spec, epochs=10, validation_data=validation_data)
```

### 导出为不同格式

导出格式可以是以下列表中的一个或多个：

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.SAVED_MODEL`


默认情况下，它仅导出包含模型[元数据](https://www.tensorflow.org/lite/models/convert/metadata)的 TensorFlow Lite 模型文件，以便以后在设备端机器学习应用中使用。标签文件嵌入在元数据中。

在许多设备端机器学习应用中，模型大小是一个重要因素。因此，建议您量化模型以使其更小并可能加快运行速度。对于 EfficientDet-Lite 模型，默认使用全整数量化来量化模型。请参阅[训练后量化](https://www.tensorflow.org/lite/performance/post_training_quantization)了解详细信息。

```python
model.export(export_dir='.')
```

您还可以选择导出与模型相关的其他文件，以便更好地进行检查。例如，按如下方式同时导出保存的模型和标签文件：

```python
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])
```

### 在 TensorFlow Lite 模型上自定义训练后量化

[训练后量化](https://www.tensorflow.org/lite/performance/post_training_quantization)是一种转换技术，可以缩减模型大小并缩短推断延迟，同时改善 CPU 和硬件加速器推断速度，且几乎不会降低模型准确率。因此，它被广泛用于优化模型。

Model Maker 库在导出模型时会应用默认的训练后量化技术。如果您想自定义训练后量化，Model Maker 也支持使用 [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig) 的多个训练后量化选项。我们以 float16 量化为例。首先，定义量化配置。

```python
config = QuantizationConfig.for_float16()
```

然后，我们使用此配置导出 TensorFlow Lite 模型。

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# 阅读更多

您可以阅读我们的[目标检测](https://www.tensorflow.org/lite/examples/object_detection/overview)示例以了解技术细节。如需了解更多信息，请参阅：

- TensorFlow Lite Model Maker [指南](https://www.tensorflow.org/lite/models/modify/model_maker)和 [API 参考](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker)。
- Task Library：用于部署的 [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector)。
- 端到端参考应用： [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android)、 [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/ios) 和 [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/raspberry_pi)。
